_____________________________________________________________________________________________________________

In [1]:
import os
os.chdir('..')

from Src.config import configuration
cfg = configuration.config_manager().get_path()

print(cfg)

folder : c:\Users\yanni\Documents\Projects\Pred_maint_projects
[2024-02-08 22:25:55,666: INFO: common: yaml file: Config\path.yaml loaded successfully]
[2024-02-08 22:25:55,666: INFO: common: created directory at: workdir]
[2024-02-08 22:25:55,666: INFO: common: created directory at: workdir/model]
{'config_path': {'work_dir': 'workdir', 'model_dir': 'workdir/model', 'local_data_file': 'data/predictive_maintenance.csv', 'local_data_file_analyse': 'workdir/predictive_maintenance_A.csv', 'local_data_file_validation': 'workdir/predictive_maintenance_validation.csv', 'local_data_file_train': 'workdir/predictive_maintenance_train.csv', 'local_data_file_test': 'workdir/predictive_maintenance_test.csv'}}


In [2]:
# Import the libraries
import pandas as pd
from Src.utils.Preprocess import utils_preprocess as preprocess

# Read the data
df = pd.read_csv(cfg.config_path.local_data_file_analyse, index_col='UDI')

# Drop the columns with no failure
#df = df.drop(df[df['Target'] == 0].index)

# Preprocess the data
Preprocess_compute = preprocess(df_=df, cfg_path=cfg.config_path)
df_train, df_test, dict_target = Preprocess_compute.preprocess(chx_target=True, chx_smote=False)


           Type Air temperature [K] Process temperature [K]  \
type     object             float64                 float64   
min           H               295.3                   305.7   
max           M               304.5                   313.8   
list  [M, L, H]                                               

     Rotational speed [rpm] Torque [Nm] Tool wear [min]  \
type                  int64     float64           int64   
min                    1168         3.8               0   
max                    2886        76.6             253   
list                                                      

     Difference temperature [K]    Power  
type                    float64  float64  
min                         7.6  10966.8  
max                        12.1  99980.4  
list                                      


In [3]:
df_train.shape


(6382, 15)

In [4]:
df_train.head()

,cat__Type_H,cat__Type_L,cat__Type_M,num__Air temperature [K],num__Process temperature [K],num__Rotational speed [rpm],num__Torque [Nm],num__Tool wear [min],num__Difference temperature [K],num__Power,Heat Dissipation Failure,No Failure,Overstrain Failure,Power Failure,Tool Wear Failure
0,0.0,1.0,0.0,0.005100,-0.134547,-0.919764,-0.025833,-0.030731,-0.210681,-0.531757,False,True,False,False,False
1,0.0,1.0,0.0,0.855059,0.605228,-0.740696,0.647203,1.578133,-0.811874,0.521683,False,False,False,False,True
2,0.0,0.0,1.0,0.305086,-0.134547,-0.512793,0.300788,-1.497636,-0.811874,0.219477,False,True,False,False,False
3,0.0,1.0,0.0,-0.344883,0.336219,-1.413555,1.973480,-1.450317,1.192102,1.628567,False,True,False,False,False
4,0.0,0.0,1.0,0.505076,0.605228,1.212764,-1.550061,1.136484,-0.110482,-1.658660,False,True,False,False,False


_______________________________________________________________________________________________________________________

### Import

In [ ]:
import pandas as pd
import numpy as np
import json
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split


In [ ]:
import warnings

warnings.filterwarnings(action='ignore')

### Read file data

In [ ]:
df = pd.read_csv(cfg.config_path.local_data_file_analyse, index_col='UDI')
df.head()

Delete rows with 'Random Failures' in type column with target == 0

In [ ]:
df_tmp = df[df['Target']==0]
df_tmp['Failure Type'].value_counts()

In [ ]:
idx = df_tmp[df_tmp['Failure Type'] == 'Random Failures'].index
df.drop(index=idx, axis=0, inplace=True)

Delete rows with 'No Failure' in type column with target == 1

In [ ]:
df_tmp = df[df['Target']==1]
df_tmp['Failure Type'].value_counts()

In [ ]:
idx = df_tmp[df_tmp['Failure Type']=='No Failure'].index
df.drop(index=idx, axis=0, inplace=True)

In [ ]:
df.shape

# Data Formatting
## Conversion 'Failure Type' variable to discret value

In [ ]:
lst_failure = df['Failure Type'].unique()
dict_failure = {}
for i, failure in enumerate(lst_failure):
    dict_failure[failure] = i
df['Failure Type'].map(dict_failure)  

## List of categorical variables
Memorize features who are 'object' type, exclude target variables 

In [ ]:
cat_cols = df.select_dtypes(include='O').columns.to_list()
cat_cols.remove('Failure Type')
#cat_cols.append('Target')
cat_cols

## List of numerical variables
Memorize features who are 'numerical' type, exclude target variables 

In [ ]:
num_cols = df.select_dtypes(exclude='O').columns.to_list()
num_cols.remove('Target')
num_cols

## Preprocessing of variables

#### Pipeline creation

Define different preprocessing to dataset
1. OnehotEncoder process from list of variables in 'cat_cols'
2. StandardScaler process from list of variables in 'num_cols'
3. OrdinalEncoder process for 'Failure Type' variables

In [ ]:
ct = ColumnTransformer([
    ("onehot", OneHotEncoder(), cat_cols),
    ("scale", StandardScaler(), num_cols),
    ("fail_type", OrdinalEncoder(), ['Target'])
])


Apply preprocess transformer and create dataframe 'df_preprocess' 

In [ ]:
df.drop(columns=['Target'], inplace=True)
df.rename(columns={'Failure Type': 'Target'}, inplace=True)
df_preprocess = pd.DataFrame(ct.fit_transform(df), columns=ct.get_feature_names_out())


In [ ]:

df_preprocess.dropna(axis=0, inplace=True)

Create dictionnary with index and value of 'target' feature

In [ ]:
cat_=ct.named_transformers_['fail_type'].categories_
dict_target = {}
dict_target_inv = {}
for i, c in enumerate(cat_[0]):
    dict_target[c] = i
    dict_target_inv[i] = c


Split dataframe into dataset for training and dataset for validation

In [ ]:
X_, X_val, y_, y_val = train_test_split(
    df_preprocess.drop(columns=['fail_type__Target']), 
    df_preprocess['fail_type__Target'], 
    test_size=0.2,
    stratify=df_preprocess['fail_type__Target'],
    random_state=42)

In [ ]:
df_train = pd.concat([X_, y_], axis=1)
df_val = pd.concat([X_val, y_val], axis=1)

Save dataframe

In [ ]:
df_train.to_csv('../data/predictive_maintenance_preprocess.csv')
df_val.to_csv('../data/predictive_maintenance_validation.csv')

Save dictionary of target

In [ ]:
with open('../data/dict_target.json', 'w') as f:
    json.dump(dict_target_inv, f)